In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image


from tyssue import config, Sheet, SheetGeometry, History, EventManager
from tyssue.draw import sheet_view
from tyssue.generation import three_faces_sheet
from tyssue.draw.plt_draw import plot_forces

from tyssue.dynamics import PlanarModel

from tyssue.solvers.viscous import EulerSolver
from tyssue.draw.plt_draw import create_gif


geom  = SheetGeometry
model = PlanarModel

# The  `history` object

The `history` object is in charge of storing the evolving epithelium during the course of the simulation. It allowed to access to different time point of a simulation from one unique object. 

Most of the time, we use `HistoryHDF5` class, that writes each time step to a file, which can be usefull for big files. It is also possible to read an hf5 file to analyse simulation later. 


In the solver, we use the `history.record` method to store the epithelium.In the `create_gif` function, we use the `history.retrieve` method to get back the epithelium at a given time point. Please see the API for more details.



# Quasistatic solver

A common way to describe an epithelium is with the **quasistatic approximation**: we assume that at any point in time, without exterior perturbation, the epithelium is at an **energy minimum**. For a given expression of the model's Hamiltonian, we thus search the position of the vertices corresponding to the minimum of energy.

## The Farhadifar model

A very common formulation for the epithelium mechanical energy was given in [Farhadifar et al. in 2007](https://doi.org/10.1016/j.cub.2007.11.049).

The energy is the sum of an area elasticity, a contractility and a line tension:
$$
E = \sum_\alpha \frac{K_A}{2}(A_\alpha - A_0)^2 + \sum_\alpha \Gamma P_\alpha^2 + \sum_{ij} \Lambda \ell_{ij} 
$$

In tyssue, `Model` objects provide functions to compute the energy for a given epithelium,  as well as it's spatial derivatives over the vertices positions, the gradient.


The calculus is done with the columns of the dataframes, for exemple, if we set the value 0.12 for the term $\Lambda$ for all the edges of the epithelium:



In [ ]:
sheet.edge_df["line_tension"] = 0.12
E_lt = (sheet.edge_df["length"] * sheet.edge_df["line_tension"]).sum()
print("Line tension energy : ", E_lt)

This model has been predifined in tyssue as the default `PlanarModel`. We can use it in combination with a `Solver` object to find the energy minimum of our sheet.

To add the various columns in the dataframes (as we did above) we use a  **specification dictionnary** with the required parameters.

In [ ]:
#from tyssue.config.dynamics import quasistatic_plane_spec
from tyssue.dynamics.planar_vertex_model import PlanarModel as smodel
from tyssue.solvers import QSSolver
from pprint import pprint

specs = {
    'edge': {
        'is_active': 1,
        'line_tension': 0.12,
        'ux': 0.0,
        'uy': 0.0,
        'uz': 0.0
    },
   'face': {
       'area_elasticity': 1.0,
       'contractility': 0.04,
       'is_alive': 1,
       'prefered_area': 1.0},
   'settings': {
       'grad_norm_factor': 1.0,
       'nrj_norm_factor': 1.0
   },
   'vert': {
       'is_active': 1
   }
}


# Update the specs (adds / changes the values in the dataframes' columns)
sheet.update_specs(specs)

pprint(specs)

In [ ]:
E_t = smodel.compute_energy(sheet)
print("Total energy: ", E_t)

In [ ]:
smodel.compute_gradient(sheet).head()

The energy minimum is found with a _gradient descent_ strategy, the vertices are displaced in the direction opposit to the spatial derivative of the energy. Actually, this defines the **force** on the vertices.


In [ ]:
from tyssue.draw.plt_draw import plot_forces

fig, ax = plot_forces(sheet, sgeom, smodel, ['x', 'y'], 1)

fig.set_size_inches(10, 10)
ax.set_aspect('equal')

In [ ]:
# Find energy minimum
solver = QSSolver()
res = solver.find_energy_min(sheet, sgeom, smodel)

print("Successfull gradient descent? ", res['success'])
fig, ax = sheet_view(sheet, ax=ax)

fig.set_size_inches(10, 10)
ax.set_aspect('equal')

fig

In [ ]:
fig, ax = sheet_view(sheet)

# Euler solver

This notebooks demonstrates usage of the time dependant solver `EulerSolver` in the simplest case where we solve
$$\eta_i \frac{d\mathbf{r}_i}{dt}  = \mathbf{F}_i = - \mathbf{\nabla}_i E$$

The model is defined in the same way it is defined for the quasistatic solver.

## Simple forward Euler solver

$$\mathbf{r}_i(t+dt) = \mathbf{r}_i(t) + \frac{\mathbf{F}_i(t)}{\eta} dt$$


In [ ]:
sheet = Sheet('3', *three_faces_sheet())
geom.update_all(sheet)
sheet.settings['threshold_length'] = 1e-3

sheet.update_specs(config.dynamics.quasistatic_plane_spec())
sheet.face_df["prefered_area"] = sheet.face_df["area"].mean()
history = History(sheet) #, extra_cols={"edge":["dx", "dy", "sx", "sy", "tx", "ty"]})

sheet.vert_df['viscosity'] = 1.0
sheet.edge_df.loc[[0, 17],  'line_tension'] *= 4
sheet.face_df.loc[1,  'prefered_area'] *= 1.2

fig, ax = plot_forces(sheet, geom, model, ['x', 'y'], 1)

Solver instanciation

In [ ]:
solver = EulerSolver(
    sheet,
    geom,
    model,
    history=history,
    auto_reconnect=True)


The solver's `solve` method accepts a `on_topo_change` function as argument. This function is executed each time a topology change occurs. Here, we reste the line tension to its original value.

In [ ]:
def on_topo_change(sheet):
    print("reseting tension")
    sheet.edge_df["line_tension"] = sheet.specs["edge"]["line_tension"]


### Solving from $t = 0$ to $t = 15$

In [ ]:
res = solver.solve(tf=15, dt=0.05, on_topo_change=on_topo_change,
                   topo_change_args=(solver.eptm,))

## Showing the results

In [ ]:
create_gif(solver.history, "sheet3.gif", num_frames=120)

In [ ]:
Image("sheet3.gif")


In [ ]:
solver = QSSolver()

scale = 0.02
fig, ax = plot_forces(sheet, geom, model, ['z', 'x'], scale)
fig.set_size_inches(10, 12)
for n, (vx, vy, vz) in sheet.vert_df[sheet.coords].iterrows():
    shift = 0.6 * np.sign(vy)
    ax.text(vz+shift-0.3, vx, str(n))

app_grad_specs = config.draw.sheet_spec()['grad']
app_grad_specs.update({'color':'r'})
    
def draw_approx_force(ax=None):
    fig, ax = plot_forces(sheet, geom, model,
                          ['z', 'x'], scaling=scale, ax=ax,
                          approx_grad=solver.approx_grad, **{'grad':app_grad_specs})
    fig.set_size_inches(10, 12)
    return fig, ax

## Uncomment bellow to recompute
fig, ax = draw_approx_force(ax=ax)
#fig

http://scipy.github.io/devdocs/generated/scipy.optimize.check_grad.html#scipy.optimize.check_grad

In [ ]:

grad_err = solver.check_grad(sheet, geom, model)
grad_err /= sheet.vert_df.size


print("Error on the gradient (non-dim, per vertex): {:.3e}".format(grad_err))


options ={'disp':False,
          'ftol':1e-5,
          'gtol':1e-5}


res = solver.find_energy_min(sheet, geom, model, options=options)
print(res['success'])

res['message']

res['fun']/sheet.face_df.is_alive.sum()